# Notebook to download data from ELIA

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime

import polars as pl

from wind_forecast_ui.elia import EliaAPIClient, EliaDataset
from wind_forecast_ui.io import ForecastConfig, IODataManager
from wind_forecast_ui.time import last_day_start_end

### Select the data you want to download
 - `data_type`: Type of data to download (e.g., 'solar', 'wind')
 - `start_date`: Start date for the data in 'YYYY-MM-DD' format
 - `end_date`: End date for the data in 'YYYY-MM-DD' format

In [ ]:
existing_data = pl.read_parquet("./data/wind/wind_target.parquet")
existing_data

In [ ]:
existing_data.filter(pl.col("datetime") >= datetime.datetime(2025, 1, 1, 0, 0, 0, tzinfo=datetime.UTC))

In [ ]:
start, end = last_day_start_end("CET")
start = datetime.datetime(2020, 1, 1, 0, 0, 0, tzinfo=datetime.UTC)
# start = max(existing_data["datetime"].max(), start)

In [ ]:
client = EliaAPIClient(EliaDataset.SOLAR_HISTORY)
df = client.load_history_measurements(start, end)

In [ ]:
df

In [ ]:
df = (
    df.rename({"value": "measured"})
    .with_columns(pl.col("datetime").dt.cast_time_unit("ms"))
    # .with_columns(pl.col("measured").truediv("monitoredcapacity").alias("loadfactor"))
    .sort("datetime", descending=False)
    # .with_columns(
    #     pl.col("decrementalbidid").str.replace(",", "").replace("''", 0.0).cast(pl.Float64).truediv(1e3),
    # )
    # .with_columns(
    #     pl.when(pl.col("decrementalbidid") > 0)
    #     .then(-pl.col("decrementalbidid"))
    #     .otherwise(pl.col("decrementalbidid"))
    #     .alias("decrementalbidid")
    # )
    # could be shifted by 1 hour or scaled differently
    # .with_columns((pl.col("measured") - pl.col("decrementalbidid")).alias("measured-decrementalbid"))
    .drop_nulls(subset=["measured"])
)

In [ ]:
df = pl.concat([existing_data, df], how="vertical").sort("datetime", descending=False)

In [ ]:
df.write_parquet("./data/solar/target.parquet")

In [ ]:
io_manager = IODataManager(ForecastConfig.SOLAR)
io_manager

In [ ]:
io_manager.update_history_benchmark_file(
    start=datetime.datetime(2025, 1, 1, 0, 0, 0, tzinfo=datetime.UTC),
    end=datetime.datetime.now(datetime.UTC) - datetime.timedelta(days=1),
)

In [ ]:
datetime.datetime.utcnow()